In [1]:
from gurobipy import Model, GRB, quicksum

In [3]:
# Sets
M = range(1, m + 1)  # Sutun Seti (columns): 1 to m
N = range(1, n + 1)  # Satır Seti (rows): 1 to n
K = range(1, w + 1)  # Dalga Seti (waves): 1 to w

# Parameter for the number of vehicles per wave
v = {
    1: 8,
    2: 9,
    3: 10,
    4: 5,
    5: 10,
    6: 10,
    7: 6,
    8: 9,
    9: 9,
    10: 36
}

# Set of vehicles (L) depends on wave (k)
L = {k: range(1, v[k] + 1) for k in K}

# Composite set A: All combinations of (i, j, k, l)
A = [(i, j, k, l) for i in M for j in N for k in K for l in L[k]]

In [4]:
# Create the Gurobi model
model = Model("AMPL_Model")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-05


In [5]:
# Decision Variables
# Binary variable x[i, j, k, l]: Indicates if vehicle l in wave k is placed in cell (i, j)
x = model.addVars(A, vtype=GRB.BINARY, name="x")

# Binary variable y[k]: Indicates if wave k is activated
y = model.addVars(K, vtype=GRB.BINARY, name="y")

# Binary variable z[i, j, k]: Auxiliary variable for composite conditions
z = model.addVars(M, N, K, vtype=GRB.BINARY, name="z")

# Continuous variable r[i, j]: Represents weighted sum of variables (auxiliary)
r = model.addVars(M, N, vtype=GRB.CONTINUOUS, lb=0, name="r")


In [6]:
# Objective: Maximize the number of waves activated
model.setObjective(quicksum(y[k] for k in K), GRB.MAXIMIZE)

In [7]:
# Constraints

# Constraint 1: Each cell can have at most one vehicle
model.addConstrs(
    (quicksum(x[i, j, k, l] for k in K for l in L[k]) <= 1 for i in M for j in N),
    "Const1"
)

# Constraint 12: Each vehicle can occupy at most one position
model.addConstrs(
    (quicksum(x[i, j, k, l] for i in M for j in N) <= 1 for k in K for l in L[k]),
    "Const12"
)

# Constraint 2: A wave is loaded if all its vehicles are loaded
model.addConstrs(
    (quicksum(x[i, j, k, l] for i in M for j in N) >= y[k] for k in K for l in L[k]),
    "Const2"
)

# Constraint 22: Waves should load sequentially
model.addConstrs(
    (y[k] >= y[k + 1] for k in range(1, w)),
    "Const22"
)

# Constraint 3: The first vehicle of the first wave must be placed in the first cell
model.addConstr(
    (quicksum(x[i, 1, 1, 1] for i in M) == 1),
    "Const3"
)

# Constraint 4: Placement rules for subsequent waves
model.addConstrs(
    (quicksum(x[i, j, k, l] for l in L[k]) +
     (1 / (j - 1)) * quicksum(x[i, a, k2, l] for k2 in range(k + 1, w + 1) for l in L[k2] for a in range(1, j)) <= 1
     for i in M for j in range(2, n + 1) for k in range(1, w)),
    "Const4"
)

# Constraint 5: Balance vehicles around the middle rows (upper bound)
model.addConstrs(
    (quicksum(x[i, j, a, l] * ((m / 2) + 1 - i) for i in range(1, int(m / 2) + 1) for j in N for a in range(k, w + 1) for l in L[a]) -
     quicksum(x[i, j, a, l] * (i - (m / 2)) for i in range(int(m / 2) + 1, m + 1) for j in N for a in range(k, w + 1) for l in L[a]) <= 2
     for k in K),
    "Const5"
)

# Constraint 6: Balance vehicles around the middle rows (lower bound)
model.addConstrs(
    (quicksum(x[i, j, a, l] * ((m / 2) + 1 - i) for i in range(1, int(m / 2) + 1) for j in N for a in range(k, w + 1) for l in L[a]) -
     quicksum(x[i, j, a, l] * (i - (m / 2)) for i in range(int(m / 2) + 1, m + 1) for j in N for a in range(k, w + 1) for l in L[a]) >= -2
     for k in K),
    "Const6"
)

# Constraint 7: Define z as a sum over x
model.addConstrs(
    (quicksum(x[i, j, k, l] for l in L[k]) == z[i, j, k] for i in M for j in N for k in K),
    "Const7"
)

# Constraint 8: Define r as a weighted sum of z
model.addConstrs(
    (quicksum(k * z[i, j, k] for k in K) == r[i, j] for i in M for j in N),
    "Const8"
)

# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2216 rows, 11818 columns and 647152 nonzeros
Model fingerprint: 0x021fb337
Variable types: 96 continuous, 11722 integer (11722 binary)
Coefficient statistics:
  Matrix range     [7e-02, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 900 rows and 1015 columns
Presolve time: 1.02s
Presolved: 1316 rows, 10803 columns, 554894 nonzeros
Variable types: 0 continuous, 10803 integer (10783 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 9.555556e+00, 1125 iterations, 0.41 secon

In [10]:
# Display the values of r in a tabular format
if model.status == GRB.OPTIMAL:
    print("r [*,*] (tr)")
    print(":    ", end="")
    
    # Print the column headers (1 to n)
    for j in N:
        print(f"{j:>4}", end="")
    print("   :=")
    
    # Print each row of r (1 to m)
    for i in M:
        print(f"{i:<4}", end="")  # Print the row header
        for j in N:
            # Print the value of r[i, j], formatted for alignment
            print(f"{r[i, j].X:>4.0f}", end="")
        print()  # Newline at the end of the row
else:
    print("No optimal solution found. Model status:", model.status)


r [*,*] (tr)
:       1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16   :=
1      2   2   0   2   2   3   3   5   5   0   8   8   9   0   9   9
2      1   0   1   1   0   1   0   4   0   0   5   0   5   5   7   7
3      3   4   4   6   6   6   0   6   6   6   6   8   8   8   9   9
4      1   2   2   3   0   3   3   3   4   5   0   5   6   7   8   8
5      1   3   3   3   6   0   0   0   6   0   7   7   7   8   0   0
6      1   1   2   2   2   4   5   5   5   8   0   9   9   9   0   9
